
# Tutoriel - Python Geoparsing 

Ce tutoriel reprend ceux proposés pour l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org) et pour la session [Natural Language Processing (NLP) for historical texts](https://github.com/ludovicmoncla/SunoikisisDC-Summer2022-Session9) de la formation [SunoikisisDC Summer 2022 Course](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9).

## 1. En bref


Dans ce tutoriel, nous allons apprendre plusieurs choses :

- Charger des jeux de données :
  - à partir de fichiers txt importés depuis le disque dur ;
  - à partir de la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) dans un [Pandas dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (articles encyclopédiques et descriptions de randonnées).
- Manipuler et interroger un dataframe
- Utiliser les librairies [Stanza](https://stanfordnlp.github.io/stanza/index.html), [spaCy](https://spacy.io) et [Perdido](https://github.com/ludovicmoncla/perdido) pour la reconnaissance d'entités nommées
  - afficher les entités nommées annotées ;
  - comparer les résultats de `Stanza`, `spaCy` et `Perdido` ;
  - discuter les dimites des 3 outils pour la tâche de NER.
- Utiliser la librarie `Perdido` pour le geoparsing et le geocoding :
  - cartographier les lieux geocodés ;
  - illustrer la problématique de désambiguïsation des toponymes.

## 2. Configurer l'environnement


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 2.2.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter les cellules suivantes :

### 2.1 Installer les librairies Python (uniquement si vous n'avez pas configuré l'environnement Python)

In [ ]:
! pip install perdido==0.1.29
! pip install stanza==1.4.2

* Télécharger les données :

In [ ]:
! mkdir data
! wget https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/data/edda-volume01-4083.txt -O data/edda-volume01-4083.txt
! wget https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/data/edda-volume02-1365.txt -O data/edda-volume02-1365.txt
! wget https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/data/lge-beaufort.txt -O data/lge-beaufort.txt

### 2.2 Importer les librairies


Tout d'abord, nous allons charger certaines bibliothèques spécifiques de `Perdido` que nous utiliserons dans ce notebook. Ensuite, nous importons quelques outils qui nous aideront à analyser et à visualiser le texte.

In [4]:
import warnings
warnings.filterwarnings('ignore')

from perdido.geoparser import Geoparser
from perdido.geocoder import Geocoder

from perdido.datasets import load_edda_artfl, load_edda_perdido, load_choucas_perdido

from spacy import displacy

## 3. Chargement et exploration des données

### 3.1 Chargement d'un document texte à partir d'un fichier


In [5]:
filepath = 'data/edda-volume01-4083.txt'
with open(filepath) as f:
    arques = f.read()

* Afficher le contenu du fichier

In [6]:
print(arques)

* ARQUES, (Géog.) petite ville de France, en Normandie, au pays de Caux, sur la petite riviere d'Arques. Long. 18. 50. lat. 49. 54.


### 3.2 Chargement d'un jeu de données à partir de la librairie Perdido

La libraire de geoparsing [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

 Dans un premier temps nous allons nous intéresser au jeu de données des articles encyclopédiques. Ce jeu de données est présent dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). Nous allons charger la version brute et voir comment manipuler un dataframe.

* Charger le jeu de données :

In [7]:
dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']

* Afficher les informations sur le jeu de données :

In [8]:
data_artfl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3385 entries, 0 to 3384
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   filename   3385 non-null   object
 1   volume     3385 non-null   int64 
 2   number     3385 non-null   int64 
 3   head       3384 non-null   object
 4   normClass  3384 non-null   object
 5   author     3384 non-null   object
 6   text       3385 non-null   object
dtypes: int64(2), object(5)
memory usage: 185.2+ KB


* Afficher les premières lignes du dataframe :

In [17]:
data_artfl.head()

,filename,volume,number,head,normClass,author,text
0,volume07-1.tei,7,1,Title Page,unclassified,unsigned,"ENCYCLOPÉDIE, ou DICTIONNAIRE RAISONNÉ DES SCI..."
1,volume07-10.tei,7,10,FOESNE ou FOUANE,Marine | Pêche,Bellin,"FOESNE ou FOUANE, sub. s. (Marine & Pêche.) c'..."
2,volume07-100.tei,7,100,Fond de la hune,unclassified,Bellin,Fond de la hune ; ce sont les planches qu on p...
3,volume07-1000.tei,7,1000,Fronteau,Bourrelier | Sellier,Diderot,"* Fronteau, terme de Sellier-Bourrelier ; c'es..."
4,volume07-1001.tei,7,1001,FRONTIERE,Géographie,Diderot,"* FRONTIERE, s. f. (Géog.) se dit des limites,..."


### 3.3 Manipulation d'un dataframe

Nous avons maintenant accès à tous les attributs et méthodes de l'objet [dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). Par exemple, nous pouvons facilement connaître le nombre de lignes dans notre dataframe qui correspond au nombre d'articles dans notre corpus :

In [9]:
n = data_artfl.shape[0]
print('Il y a ' + str(n) + ' articles dans le jeu de données.')

Il y a 3385 articles dans le jeu de données.


#### 3.3.1 Recherche par métadonnées


Maintenant que les données sont chargées dans un dataframe, nous pouvons sélectionner des groupes d'articles sur la base de leurs métadonnées.

Pour cela on utilise la méthode [loc()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html).



* Selectionner la ligne du dataframe qui correspond à l'article 'FRONTIGNAN' :

In [10]:
frontignan = data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN']
frontignan

,filename,volume,number,head,normClass,author,text
5,volume07-1002.tei,7,1002,FRONTIGNAN,Géographie,Jaucourt,"FRONTIGNAN, (Géog.) petite ville de France. au..."


* Récupérer les valeurs des attributs (colonnes) :

In [11]:
print('volume :', frontignan.volume.item())   # similaire à frontignan['volume'].item()
print('number :', frontignan.number.item())
print('text :', frontignan.text.item())

volume : 7
number : 1002
text : FRONTIGNAN, (Géog.) petite ville de France. au Bas-Languedoc, connue par ses excellens vins muscats, & ses raisins de caisse qu'on appelle passerilles. Quelques savans croyent, sans en donner de preuves, que cette ville est le forum Domitii des Romains. Elle est située sur l'étang de Maguelone, à six lieues N. E. d'Agde, & cinq S. O. de Montpellier. Long. 15d. 24'. lat. 43d. 28'. (D. J.)


Nous pouvons également filtrer les données sur la base de l'auteur.

* Extraire les articles rédigés par Jaucourt :

In [12]:
req = 'Jaucourt'
d_Jaucourt = data_artfl.loc[data_artfl['author'] == req]

n = d_Jaucourt.shape[0]
print(str(n) + ' articles ont été rédigés par '+ req)

698 articles ont été rédigés par Jaucourt


* Afficher les 5 premiers :

In [13]:
d_Jaucourt.head()

,filename,volume,number,head,normClass,author,text
5,volume07-1002.tei,7,1002,FRONTIGNAN,Géographie,Jaucourt,"FRONTIGNAN, (Géog.) petite ville de France. au..."
29,volume07-1024.tei,7,1024,"FROWARD, le cap.",Géographie,Jaucourt,"FROWARD, le cap. (Géog.) & par les François le..."
32,volume07-1027.tei,7,1027,FRUGALITÉ,Morale,Jaucourt,"FRUGALITÉ, (Morale.) simplicité de moeurs & de..."
37,volume07-1031.tei,7,1031,Fruit verreux,Histoire naturelle,Jaucourt,"Fruit verreux, (Hist. nat.) c'est le nom qu'on..."
38,volume07-1032.tei,7,1032,"Fruit, (art de conserver le)",Economie rustique,Jaucourt,"Fruit, (art de conserver le) Economie rustiq. ..."


### 3.3.2 Recherche par mots-clés

* Récupérer la liste des articles contenant l'expression "ville de" :

In [16]:
req = 'ville de'
d_res = data_artfl[data_artfl['text'].str.contains(req, case=False)]

n = d_res.shape[0]
print(str(n) + " articles contiennent l'expression '"+ req + "'")

d_res.head()

177 articles contiennent l'expression 'ville de'


,filename,volume,number,head,normClass,author,text
5,volume07-1002.tei,7,1002,FRONTIGNAN,Géographie,Jaucourt,"FRONTIGNAN, (Géog.) petite ville de France. au..."
82,volume07-1072.tei,7,1072,FUM-CHIM,Géographie,Jaucourt,"FUM-CHIM, (Géog.) petite ville de la province ..."
104,volume07-1092.tei,7,1092,FUNCHAL,Géographie,Jaucourt,"FUNCHAL, (Géog.) ville de l'Océan atlantique, ..."
114,volume07-1100.tei,7,1100,Funérailles des Romains,unclassified,Jaucourt,Funérailles des Romains. Les Romains ont eté s...
129,volume07-1114.tei,7,1114,FUNG,Géographie,Jaucourt,"FUNG, (Géog.) ville de la Chine, dans la provi..."


Dans cette partie nous avons vu brievement comment manipuler un dataframe pour selectionner certaines données en filtrant selon certaines métadonnées ou par une recheche par mot clés. Ces opérations sont utiles mais un peu limitées, nous allons voir dans la suite de ce notebook comment enrichir les métadonnées et en particulier comment annoter les entités nommées présents dans les textes.

## 4. Perdido Geoparser

La reconnaissance d'entités nommées, *Named Entity Recognition* (NER) en anglais, est une tâche très importante et incontournable en traitement automatique des langues (TAL) et en compréhension du langage naturel (NLU en anglais). 
Cette tâche consiste à rechercher des objets textuels (un mot, ou un groupe de mots, souvent associés aux noms propres) catégorisables dans des classes telles que noms de personnes, noms d'organisations ou d'entreprises, noms de lieux, quantités, distances, valeurs, dates, etc.
Les typologies et les jeux d'étiquettes sont dépendents de chaque outil.

Dans cet atelier nous allons expérimenter l'outil [Perdido](https://github.com/ludovicmoncla/perdido).

`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le geotagging et le geocoding. Le geotagging est similaire à la tâche de reconnaissance des entités nommées avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).
Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.



### 4.1 Lancer le traitement

Dans cette partie nous allons voir comment utiliser `Perdido` pour la reconnaissance d'entités nommées toujours à partir de notre exemple `Arques`.

* Instancier et paramétrer la chaîne de traitement :

In [ ]:
geoparser = Geoparser(version="Encyclopedie")

* Executer la reconnaissance d'entités nommées :

In [ ]:
doc = geoparser(arques)

Perdido effectuant la tâche de geocoding en plus du NER, le temps de traitement est plus long qu'avec d'autres outils de NER comme par exemple Stanza ou spaCy.


### 4.2 Visualisation des résultats

* Afficher le contenu XML-TEI retourné par Perdido :

In [ ]:
doc.tei

* Afficher le contenu geojson retourné par Perdido :

In [ ]:
doc.geojson

* Afficher les annotation au format BIO

In [ ]:
for token in doc:
    print(token.tsv_format())

* Afficher la liste des entités nommées repérées :

In [ ]:
for ent in doc.named_entities:
    print(ent.text, ent.tag)

* Afficher de manière graphique les entités nommées avec `displaCy` grâce à la méthode de conversion `to_spacy_doc`:

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

* Un rendu similaire mais qui permet de visualiser les entités imbriquées (`style="ent"` -> `style="span"`) :

In [ ]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

* Afficher la carte des toponymes localisés :

In [ ]:
doc.get_folium_map()

Par défaut, lors de l'instanciation du `Geoparser()`, seul [OpenStreetMap](https://www.openstreetmap.org/) est utilisé pour le geocoding et au maximum un résultat est retourné pour chaque lieu (nous verrons dans la suite comment paramétrer le geocoding).

On a déjà ici un aperçu de la difficulté de la tâche de résolution des toponymes. En effet, un grand nombre d'ambiguïtés existent tels que plusieurs lieux ayant le même nom, plusieurs noms pour un même lieu ou encore le fait qu'un lieu ne soit pas référencé dans les ressources que l'on interroge.

### 4.3 Enregistrement des résultats

* Enregistrer le résultat XML-TEI dans un fichier XML :

In [ ]:
doc.to_xml('filename.xml')

* Enregistrer le résultat au format geoJSON :

In [ ]:
doc.to_geojson('filename.geojson')

* Enregistrer les annotations au format BIO dans un fichier TSV : 

In [ ]:
doc.to_iob('filename.tsv')

* Enregistrer les résultats du NER dans un fichier CSV :

In [ ]:
doc.to_csv('filename.csv')

## 5. Perdido Geocoder

En complément du `Geoparser` qui prend en paramètre un texte et qui fait la reconnaissance d'entités nommées en amont de l'étape de geocoding, `Perdido` propose également une fonction de geocoding disctincte prenant en paramètre directement un nom de lieu (ou une liste de noms de lieux).

In [ ]:
geocoder = Geocoder()
doc = geocoder(['Arques', 'France', 'Normandie', 'Caux'])

# afficher la carte des lieux localisés
doc.get_folium_map()

### 5.1 Résolution de toponymes / désambiguïsation


* Cherchons à localiser la ville `Arques`


In [ ]:
geocoder = Geocoder()
doc = geocoder('Arques')
doc.get_folium_map()

On remarque que par défaut, la localisation retournée pour le nom de lieu `Arques` n'est pas celle que l'on recherche. En effet, le texte indique qu'il s'agit d'une ville de Normandie, or ici la localisation proposée est située dans le Pas-de-Calais !

Changeons les paramètres du `Geocoder` (ces paramètres sont similaires pour le `Geoparser`) pour essayer de retrouver la bonne localisation.

* Augmenter le nombre de résultats retournés par les gazetiers interrogés

In [ ]:
geocoder = Geocoder(max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

On observe parmi les 10 localisations retournées par OpenStreetMap (gazetier par défaut) qu'aucune ne se situe en Normandie.

* Remplacer OpenStreetMap par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'])
doc = geocoder('Arques')
doc.get_folium_map()

On observe que le premier résultat retourné par l'IGN ne se situe ni en Normandie (comme attendu), ni dans le Pas-de-Calais comme le premier résultat retourné par OpenStreetMap.

* Augmenter le nombre de résultats retournés par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'], max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

Cette fois-ci on retrouve bien une localisation en Normandie au sud de Dieppe avec pour nom `Arques-la-Bataille'. On peut faire l'hypotèse que le nom a évolué car cette localisation se situe bien dans le Pays de Caux (voir illustration ci-dessous, source [Wikipedia](https://fr.wikipedia.org/wiki/Pays_de_Caux)) comme l'indique le texte de l'article.

![Pays de Caux](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Carte_pays_Caux1.png/497px-Carte_pays_Caux1.png)

Ce problème d'ambiguïtés peut aussi être illustrer en allant directement interroger le site web du [géoportail](https://www.geoportail.gouv.fr) comme le montre la capture ci-dessous.

![Résultats sur le géoportail](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/raw/main/img/geoportail-arques.png)


Il reste néanmoins le problème de retrouver cette localisation de manière automatique. 
Plusieurs approches existent dans la littérature mais ne sont pas encore implémentées dans `Perdido`.

Cet exemple illustre bien la difficulté de la problématique de désambiguïsation des toponymes avec notamment la gestion des natures de lieux différentes (pays, régions, communes, lieux-dits, lac, rivières, etc.) associés à un même nom, l'homonymie, la non exaustivité des ressources, l'évolution des noms au cours du temps ou encore les erreurs d'orthographe.

* Afficher la carte obtenue après le geoparsing avec l'IGN et 10 résultats max par nom de lieu

In [ ]:
geoparser = Geoparser(sources=['ign'], max_rows=10)
doc = geoparser(arques)
doc.get_folium_map()


### 5.2 Le cas des descriptions de randonnées

Prenons maintenant l'exemple du geoparsing de descriptions de randonnées. Certaines solutions de désambiguisation ont pu être développées et intégrées au sein de la librairie `Perdido` (d'autres sont en cours d'intégration). Les solutions décrites dans la suite de cette partie ont été développées dans le cadre des projets [Perdido](http://erig.univ-pau.fr/PERDIDO/) (2012-2015) et [ANR CHOUCAS](http://choucas.ign.fr) (2017-2022). 

> Ludovic Moncla, Walter Renteria-Agualimpia, Javier Nogueras-Iso and Mauro Gaio (2014). "Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus". In Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, pp 183-192.

> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.


Nous avons choisi un exemple pour illustrer les différentes phases du processus que nous avons mis en place dans le cadre du geoparsing de descriptions de randonnées :
1. filtrer les résultats en fonction du pays 
2. filtrer les résultats en fonction d'une zone géographique définie 
3. regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*)
4. selectionner le cluster qui contient le plus d'entités distinctes

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 
Cette stratégie est adaptée pour une description d'itinéraire où les différents lieux cités sont supposés être localisés à proximité les uns des autres.

* Charger le jeu de données CHOUCAS de descriptions de randonnées fourni par `Perdido`

In [ ]:
dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

data_choucas.to_dataframe().head()

In [ ]:
len(data_choucas)

* Sélectionner une randonnée (parmi les 30)

In [ ]:
id_rando = 2
doc = data_choucas[id_rando]

In [ ]:
doc.text

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [ ]:
doc.get_folium_map()

On observe ici le résultat déjà pré-traité par `Perdido`. Nous allons maintenant illustrer le processus de désambiguïsation.

On recommence le processus de geoparsing en entier à partir du texte de la randonnées choisie.

In [ ]:
geoparser = Geoparser()
doc_geoparsed = geoparser(doc.text)

In [ ]:
doc_geoparsed.get_folium_map()

On voit clairement la différence par rapport au résultat précédent. Nous allons alors essayer de retrouver le même résultat en déroulant les différentes étapes pour désambiguïser avec `Perdido`.

Pour gagner un peu de temps lors des prochaines executions nous allons faire directement appel à la fonction de geocoding à partir de la liste des noms de lieux.

* Récuperer la liste des noms de lieux (sans doublon)

In [ ]:
places_list = list(set([ent.text for ent in doc_geoparsed.ne_place]))
print(places_list)

#### 5.2.1 Ajout d'un filtre "code pays"


In [ ]:
# instancier le geocoder avec le code pays
geocoder = Geocoder(country_code = 'fr')
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

doc_geocoded.get_folium_map()

#### 5.2.2 Ajout d'un filtre "bounding box"

In [ ]:
bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403] # zone d'intervention du PGHM Isère

# instancier le geocoder avec le code pays et une bounding box
geocoder = Geocoder(country_code = 'fr', bbox = bbox)
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

# affiche la carte
doc_geocoded.get_folium_map()

#### 5.2.3 Clustering par densité spatiale

In [ ]:
# appliquer la désambiguïsation 
doc_geocoded.cluster_disambiguation()
doc_geocoded.get_folium_map()

## 6. Pour aller plus loin

Dans cette partie nous allons expérimenter et comparer deux autres outils de NER ([Stanza](https://stanfordnlp.github.io/stanza/index.html) et [spaCy](https://spacy.io)).

### 6.1 Stanza NER

`Stanza` est une librairie Python de traitement du langage naturel. Elle contient des outils, qui peuvent être utilisés dans une chaîne de traitement, pour convertir du texte en listes de phrases et de mots, pour générer les formes de base de ces mots, leurs parties du discours et leurs caractéristiques morphologiques, pour produire une analyse syntaxique de dépendance, et pour reconnaître les entités nommées. 

`Stanza` se base sur des modèles entrainés par des réseaux de neurones à partir de la bibliothèque [PyTorch](https://pytorch.org) et permet de traiter plus de 70 langues.

Dans cette partie nous allons voir comment utiliser `Stanza` pour la reconnaissance d'entités nommées à partir de textes en français.


* Importer la librairie `Stanza` : 

In [ ]:
import stanza

* Télécharger le modèle pré-entrainé pour le français : 

In [ ]:
stanza.download('fr')

* Instancier et paramétrer la chaîne de traitement :

In [ ]:
stanza_parser = stanza.Pipeline(lang='fr', processors='tokenize,ner')

* On utilise la variable `arques` qui contient le texte chargé précédemment à partir du fichier txt

In [ ]:
print(arques)

* Executer la reconnaissance d'entités nommées :

In [ ]:
arques_stanza = stanza_parser(arques)

* Afficher la liste des entités nommées repérées. Avec Stanza, le résultat de l'analyse est un itérateur:

In [ ]:
for ent in arques_stanza.ents:
    print(ent.text, ent.type)

### 6.2 SpaCy NER


`spaCy` est également une librairie Python de traitement du langage naturel. 
Elle se compose de modèles pré-entrainés et supporte actuellement la tokenisation et l'entrainement pour plus de 60 langues. Elle est dotée de modèles de réseaux de neuronnes pour l'étiquettage, l'analyse syntaxique, la reconnaissance d'entités nommées, la classification de textes, l'apprentissage multi-tâches avec des transformateurs pré-entraînés comme BERT, ainsi qu'un système d'entraînement prêt pour la production et un déploiement simple des modèles. `spaCy` est un logiciel commercial, publié en open-source sous la licence MIT.

Dans cette partie nous allons voir comment utiliser `spaCy` pour la reconnaissance d'entités nommées toujours à partir de notre exemple en français.

* Installer le modèle français pré-entrainé de `spaCy` :

In [ ]:
!python -m spacy download fr_core_news_sm

* Importer la librarie `spaCy` :

In [ ]:
import spacy

* Charger le modèle français pré-entrainé de `spaCy`

In [ ]:
spacy_parser = spacy.load('fr_core_news_sm')

* Executer la reconnaissance d'entités nommées :

In [ ]:
arques_spacy = spacy_parser(arques)

* Afficher la liste des entités nommées repérées. Les sorties de SpaCy sont dans un format similaire à celui de Stanza mais les étiquettes sont portées par l'attribut `label_` et pas `type`:£

In [ ]:
for ent in arques_spacy.ents:
    print(ent.text, ent.label_)

* `spaCy` fournit également une fonction pour effectuer un rendu plus graphique des annotations avec `displaCy` :

In [ ]:
displacy.render(arques_spacy, style="ent", jupyter=True) 

On remarque des différences entre les résultats de Stanza et de spaCy. En particulier spaCy repère trois entités à tord (faux positifs) : `Géog`, `Long` et `lat`, là où Stanza ne repérait à tord que `Géog)`. Et spaCy ne repère pas la première occurrence `ARQUES` sans doute du au fait que le mot est en majuscule.

### 6.3 Expérimentations et comparaison

* Charger l'article `Beaufort` (volume 2, numéro 1365) disponible dans le dossier `data` :

In [ ]:
beaufort = load_txt('data/edda-volume02-1365.txt')

print(beaufort)

* Perdido

In [ ]:
beaufort_perdido = geoparser(beaufort)
displacy.render(beaufort_perdido.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

* spaCy

In [ ]:
beaufort_spacy = spacy_parser(beaufort)
displacy.render(beaufort_spacy, style="ent", jupyter=True) 

Dans cet exemple, `spaCy` repère le mot `Oron` comme une entité de personne alors que `Perdido` le repère comme un lieu.
On observe qu'il manque l'accent au mot «rivière». Corrigeons le texte pour voir s'il est possible d'améliorer la reconnaissance.


In [ ]:
normalized_beaufort = beaufort.replace('riviere', 'rivière')

normalized_beaufort_spacy = spacy_parser(normalized_beaufort)

displacy.render(normalized_beaufort_spacy, style="ent", jupyter=True) 


Ce changement ne corrige pas l'erreur d'annotation, au contraire l'entité n'est même plus repérée. Cependant, on observe également un saut de ligne entre les mots «rivière» et «d'Oron».
Ce retour à la ligne est due à la largeur de la colonne dans l'œuvre originale. 


![beaufort](https://github.com/ludovicmoncla/tutoriel-geoparsing/blob/main/img/beaufort_originale.png?raw=true)


Pour vérifier l'hypothèse que ce retour perturbe le repérage par `spaCy`, corrigeons une nouvelle fois le texte.


In [ ]:
normalized_beaufort = normalized_beaufort.replace('\n', '')

normalized_beaufort_spacy = spacy_parser(normalized_beaufort)

displacy.render(normalized_beaufort_spacy, style="ent", jupyter=True) 

Cette fois l'entité étendue incluant le nom commun «rivière» a été reconnu par `spaCy`, qui a pu ainsi corriger le type de l'entité nommée et se rendre compte que l'Oron était un lieu et pas une personne.

Essayons maintenant avec `Stanza`.

- Stanza

In [ ]:
beaufort_stanza = stanza_parser(beaufort)
for ent in beaufort_stanza.ents:
    print(ent.text, ent.type)

Stanza a directement repéré que l'Oron était un lieu mais veut, comme SpaCy, annoter «Géog» qui ne devrait pas l'être.

Regardons maintenant ce que l'on dit sur la même ville de Beaufort un peu plus d'un siècle plus tard, fin XIXème siecle, dans [La Grande Encyclopédie](https://www.collexpersee.eu/projet/disco-lge/) (LGE).

In [ ]:
lge_beaufort = load_txt('data/lge-beaufort.txt')
print(lge_beaufort)

Cette fois l'article est un peu plus long et comporte des césures de lignes importantes, définissons donc une fonction pour recoller les morceaux :

In [ ]:
def join_lines(s):
    return s.replace('¬\n', '').replace('-\n', '').replace('\n', ' ')

In [ ]:
normalized_lge_beaufort = join_lines(lge_beaufort)
normalized_lge_beaufort

* Perdido

lge_beaufort_perdido = geoparser(normalized_lge_beaufort)
displacy.render(lge_beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

* spaCy

In [ ]:
lge_beaufort_spacy = spacy_parser(normalized_lge_beaufort)
displacy.render(lge_beaufort_spacy, style="ent", jupyter=True)

* Stanza

In [ ]:
lge_beaufort_stanza = stanza_parser(normalized_lge_beaufort)
for ent in lge_beaufort_stanza.ents:
    print(ent.text, ent.type)

Quelques observations : 
1. Seul Perdido repère la date (1841).
2. spaCy ne classe pas correctement Albertville (Personne) contrairement à Perdido et Stanza (Lieu), spaCy ne repère pas l'entité Heni IV contrairement à Perdido et Stanza.
3. Stanza repère et classe correctement l'entité "Saint-Maximede-Bf.aufort", Perdido la repère mais ne sait pas la classer et spaCy ne la repère pas.